In [ ]:
#pytorchによる学習

In [5]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader 
from torch import nn
import torch.optim as optim
from torchvision.datasets import MNIST, CIFAR10, FashionMNIST
from torchvision.datasets import CIFAR10

import numpy as np
#import pandas as pd
import sys
import time
from tqdm import tqdm
import os

from dataclasses import dataclass

In [6]:
import dsdl

ds = dsdl.load('a9a')
print(ds)

X_train, t_train = ds.get_train()
X_test, t_test = ds.get_test()

print(type(X_test))

X_train = X_train.toarray()
print(X_train.dtype)
X_test = X_test.toarray()
X_test = np.block([X_test, np.zeros((16281, 1))])

print(t_train[1])
print(type(t_train[1]))

X_train = torch.from_numpy(X_train.astype(np.float32)).clone()
t_train = torch.from_numpy(t_train.astype(np.float32)).clone()
X_test = torch.from_numpy(X_test.astype(np.float32)).clone()
t_test = torch.from_numpy(t_test.astype(np.float32)).clone()

ds_train = TensorDataset(X_train, t_train) 
ds_test = TensorDataset(X_test, t_test)

loader_train = DataLoader(ds_train, batch_size = 128, shuffle = True)
loader_test = DataLoader(ds_test, batch_size = 128, shuffle = True)

print(X_train.shape)
print(X_test.shape)

#X_train = train.data.type(torch.float32)
#t_train = train.targets

#print(X_train)

{'train': '(32561, 123), (32561,)', 'val': '', 'test': '(16281, 122), (16281,)'}
<class 'scipy.sparse.csr.csr_matrix'>
float64
-1.0
<class 'numpy.float64'>
torch.Size([32561, 123])
torch.Size([16281, 123])


In [23]:
@dataclass
class configs():
    noise_type: str
    dataset: str
    learning_rate: float
    batch_size: int
    epochs: int = 10       

In [8]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
def param_xavier_init(model):
    for name, param in model.named_parameters():
        if 'weight' in name:
            nn.init.xavier_normal_(param)
        if 'bias' in name:
            nn.init.constant_(param, val=0)

In [51]:
def make_root(configs):
    root = 'logs/log_' + configs.dataset + '_' + configs.noise_type
    return root

In [11]:
def update_param(model, loss, lr):
    with torch.no_grad():
        for params in model.parameters():
            params.data -= lr * params.grad.data

In [12]:
def get_path(root):
    i = 1
    while True:
        path = root + '_' + str(i)
        is_file = os.path.isfile(path)
        if is_file:
            i += 1
        else:
            break
        
    return path

#root = 'log_mnist_finite'
#path = get_path(root)
#print(path)

In [13]:
#n_in = 28*28
n_in = 123
n_mid = 216
#n_out = 10
n_out = 1

class MFNN(nn.Module):
    def __init__(self):
        super(MFNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu = nn.Sequential(
            nn.Linear(n_in, n_mid),
            nn.ReLU(),
            nn.Linear(n_mid, n_mid),
            nn.ReLU(),
            nn.Linear(n_mid, n_out),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu(x)
        return logits/n_mid


In [14]:
def func(epoch):
    if epoch < 120:
        return 0.2**0
    elif epoch < 160:
        return 0.2**1
    elif epoch < 200:
        return 0.2**2
    else:
        return 0.2**3

In [79]:
#root = 'logs/log_mnist_sample'
config = configs('finite', 'a9a', 0.1, 128)
root = make_root(config)
path = get_path(root)

        
def main(model, opt, loss_fn, scheduler, epochs):
    for epoch in range(epochs):
        with open(path, 'a') as f:
            f.write(f'EPOCH: {epoch}\n')
        print(f'EPOCH: {epoch}')
        
        start = time.time() 
        
        #start train loop----------------------
        train_loss = []
        total_train = 0
        correct_train = 0
        
        model.train()
        for X, t in tqdm(loader_train):
            X, t = X.to(device), t.to(device)
            y = model(X)
            #print(y)
            t = t.to(torch.float32)
            y = y.to(torch.float32)
            t = t.view(-1, 1)
            loss = loss_fn(y, t)
            #print(loss)
            opt.zero_grad() 
            loss.backward()
            opt.step()
        
            pred = y.argmax(1)
            train_loss.append(loss.tolist())
            
            total_train += t.shape[0]
            correct_train += (pred==t).sum().item()
        scheduler.step()
    
        end  = time.time()
        
        log = f'train loss: {np.mean(train_loss):.3f}, accuracy: {correct_train/total_train:.3f}'
        with open(path, 'a') as f:
            f.write(log + f'time: {end - start:.5f}' + '\n')
        print(log)
        #end train loop------------------------------------
        
        #start test loop-----------------------------------
        test_loss = []
        total_test= 0
        correct_test = 0
    
        model.eval()
        for X, t in loader_test:
            X, t = X.to(device), t.to(device)
            y = model(X)
            t = t.to(torch.float32)
            y = y.to(torch.float32)
            t = t.view(-1, 1)            
            loss = loss_fn(y, t)
            pred = y.argmax(1)
            #print(pred)
            test_loss.append(loss.tolist())
            
            total_test += t.shape[0]
            correct_test += (pred==t).sum().item()
    
        log = f'test loss: {np.mean(test_loss):.3f}, accuracy: {correct_test/total_test:.3f}'
        with open(path, 'a') as f:
            f.write(log + '\n')
        print(log)
        # end test loop-------------------------------------- 
        
        print(f'time: {end - start:.5f}')           

In [80]:
#configs = configs('finite', 'a9a', 0.1, 128)

model = MFNN().to(device)
param_xavier_init(model)
opt = optim.SGD(model.parameters(), lr = 0.01, weight_decay = 0.01)
#loss_fn = nn.BCELoss()
loss_fn = nn.MSELoss()
scheduler = optim.lr_scheduler.LambdaLR(opt, lr_lambda = func, verbose=True)
epochs = 10
main(model, opt, loss_fn, scheduler, epochs)

Adjusting learning rate of group 0 to 1.0000e-02.
EPOCH: 0


100%|████████████████████████████████████████| 255/255 [00:00<00:00, 368.89it/s]


Adjusting learning rate of group 0 to 1.0000e-02.
train loss: 1.000, accuracy: 0.000
test loss: 1.000, accuracy: 0.000
time: 0.69373
EPOCH: 1


100%|████████████████████████████████████████| 255/255 [00:00<00:00, 369.93it/s]


Adjusting learning rate of group 0 to 1.0000e-02.
train loss: 1.000, accuracy: 0.000
test loss: 1.000, accuracy: 0.000
time: 0.69050
EPOCH: 2


...

Adjusting learning rate of group 0 to 1.0000e-02.
train loss: 1.000, accuracy: 0.000
test loss: 1.000, accuracy: 0.000
time: 0.67240
EPOCH: 8


100%|████████████████████████████████████████| 255/255 [00:00<00:00, 379.88it/s]


Adjusting learning rate of group 0 to 1.0000e-02.
train loss: 1.000, accuracy: 0.000
test loss: 1.000, accuracy: 0.000
time: 0.67239
EPOCH: 9


100%|████████████████████████████████████████| 255/255 [00:00<00:00, 380.41it/s]


Adjusting learning rate of group 0 to 1.0000e-02.
train loss: 1.000, accuracy: 0.000
test loss: 1.000, accuracy: 0.000
time: 0.67142
